### Import Packages

In [1]:
## import numpy
import numpy as np
np.set_printoptions(precision=6)

## import functions from twinning
import sys; sys.path.insert(0, '..')
from nmgjunctions.twinning import *

_____

_____

### Lattice Parameters, Irrational Elements, Metric Tensor, and Transformation Matrics

In [2]:
## NMG lattice parameter: Seiner et al. (2019)
a,b,c,gamma = [0.5972, 0.5944, 0.5584, np.deg2rad(90.37)]

## irrational element
q3 = (2*a*b*np.cos(gamma) - np.sqrt(a**4 + b**4 + 2*a**2*b**2*np.cos(2*gamma)))/(a**2 - b**2)
q4 = (2*a*b*np.cos(gamma) + np.sqrt(a**4 + b**4 + 2*a**2*b**2*np.cos(2*gamma)))/(a**2 - b**2)

# Metric Tensor
G = np.array([[a**2,a*b*np.cos(gamma),0],[a*b*np.cos(gamma),b**2,0],[0,0,c**2]])

## Transformation matrices obtained from previous work
rPv1, rPv3, rPv2, rPv4, rPv1s = np.load('rPv13241s.npy')

## Mirror reflection about xy-pane (or z-axis)
M_z = np.array([[1,0,0],[0,1,0],[0,0,-1]])

_____

_____

## Shear Matrix and Misorientation Relationship in Reference Frame

In [3]:
## define global variables for function
global G 

#function 1: obtain shear matrix of twin transformation in reference frame
def ShearMatrix_Ref(K1,eta1,C,rPm,normal_pos=True):
    """
    Returns Shear Matrix of twin transformation in reference frame
    
    Parameters
    ----------
    K1: numpy.ndarray(1,3), twin boundary in crystal frame
    eta1: numpy.ndarray(3,1), shear direction in crystal frame
    C: numpy.ndarray(3,1), Correspondence Matrix
    rPm: numpy.ndarray(3,3), transformation matrix (crystal frame ↔ reference frame)
    normal_pos: bool, plane normal points to the twin (default true) 
    
    Returns
    -------
    numpy.ndarray(3,3), returns Shear Matrix in reference frame
    """
    
    # convert to array
    K1 = np.squeeze(np.asarray(K1))
    eta1 = np.squeeze(np.asarray(eta1))
    rPm = np.asarray(rPm)
    
    # convert K1 and eta1 if plane normal is not positive
    if normal_pos == False:
        eta1 = -eta1
        K1 = -K1
    

    ## calculate shear
    s = np.sqrt(np.trace(C.T @ G @ C @ inv(G))-3)
    
    ## obtain plane normal (m) and shear direction (l) in R-frame:
    rm = K1 @ inv(rPm)
    rm = rm / np.sqrt(rm @ rm.T) #unit vector
    
    rl = rPm @ eta1
    rl = rl / np.sqrt(rl @ rl.T)
    
    ## shear matrix: Using einstein convention
    rS = np.zeros([3,3]) #initialize
    
    #1st row
    rS[0,0] = 1 + s * rl[0] * rm[0] #i=0,j=0
    rS[0,1] = 0 + s * rl[0] * rm[1] #i=0,j=1
    rS[0,2] = 0 + s * rl[0] * rm[2] #i=0,j=2
    #2nd row
    rS[1,0] = 0 + s * rl[1] * rm[0] #i=1,j=0
    rS[1,1] = 1 + s * rl[1] * rm[1] #i=1,j=1
    rS[1,2] = 0 + s * rl[1] * rm[2] #i=1,j=2
    #3rd row
    rS[2,0] = 0 + s * rl[2] * rm[0] #i=0,j=0
    rS[2,1] = 0 + s * rl[2] * rm[1] #i=0,j=1
    rS[2,2] = 1 + s * rl[2] * rm[2] #i=0,j=2
    
    return rS


#function 2: obtain misorientation of twin transformation in reference frame
def OR_RefFrame(K1,eta1,C,rPm,normal_pos=True):
    """
    Returns orientation relationship (OR) of twins in reference frame
    
    Parameters
    ----------
    K1: numpy.ndarray(1,3), twin boundary in crystal frame
    eta1: numpy.ndarray(3,1), shear direction in crystal frame
    C: numpy.ndarray(3,1), Correspondence Matrix
    rPm: numpy.ndarray(3,3), transformation matrix (crystal frame ↔ reference frame)
    normal_pos: bool, plane normal points to the twin (default true) 
    
    Returns
    -------
    numpy.ndarray(3,3), returns OR of NC twins in reference frame
    """
    
    ## Shear Matrix
    rS = ShearMatrix_Ref(K1,eta1,C,rPm,normal_pos)
    
    ## Orientation relationship in R-frame
    rL = rPm @ C @ inv(rPm) @ inv(rS)
    return rL

### $V_1 \to V_3$ Transformation

#### $^R\boldsymbol{L}: V_1 \to V_3$

In [4]:
## V₁ → V₃ Misorientation

## twinning elements
C =  rotationmatrix([1,1,0],np.pi) #correspondence matrix
K1 = [1,1,0] #invariant plane
eta1 = [-1,1,0] #shear direction

## Rotational transformation in reference frame: V₁ → V₃
rL_v1tov3 = OR_RefFrame(K1,eta1,C,rPv1,normal_pos=True)

## Print required outputs
print('ᴿL(V₁ → V₃) =\n', rL_v1tov3) #Orientation relationship
print('\n ᴿL(V₁ → V₃) [0,0]: %.4f == cos(%.2f deg)' % (rL_v1tov3[0,0],np.degrees(np.arccos(rL_v1tov3[0,0])))) #rotation angle
print('\n ᴿL(V₁ → V₃) [1,0]: %.4f == sin(%.2f deg)' % (rL_v1tov3[1,0],np.degrees(np.arcsin(rL_v1tov3[1,0])))) #rotation angle

ᴿL(V₁ → V₃) =
 [[-0.011127 -0.999938  0.      ]
 [-0.999938  0.011127  0.      ]
 [ 0.        0.       -1.      ]]

 ᴿL(V₁ → V₃) [0,0]: -0.0111 == cos(90.64 deg)

 ᴿL(V₁ → V₃) [1,0]: -0.9999 == sin(-89.36 deg)


_____

#### Equivalent Description of $\ ^R\boldsymbol{L}: V_1 \to V_3$

$^R\boldsymbol{L}\ \simeq\ ^R\boldsymbol{L}\ \cdot\ ^R\boldsymbol{M}_z$

where, $^R\boldsymbol{M}_z$ is the mirror operator about $[001]_R$.

In [5]:
## Equivalent description of V₁ → V₃ misorientation
## ᴿL (V₁ → V₃) ⋅ Mz
dummy = rL_v1tov3 @ M_z #Orientation relationship

## Print required output
print('ᴿL(V₁ → V₃) ⋅ Mz = \n', dummy)
print('\n ᴿL(V₁ → V₃)⋅Mz [0,0]: %.4f == cos(%.2f deg)' % (dummy[0,0],np.degrees(np.arccos(dummy[0,0])))) #rotation angle
print('\n ᴿL(V₁ → V₃)⋅Mz [1,0]: %.4f == sin(%.2f deg)' % (dummy[1,0],np.degrees(np.arcsin(dummy[1,0])))) #rotation angle

## del unnecessary variables
del dummy

ᴿL(V₁ → V₃) ⋅ Mz = 
 [[-0.011127 -0.999938  0.      ]
 [-0.999938  0.011127  0.      ]
 [ 0.        0.        1.      ]]

 ᴿL(V₁ → V₃)⋅Mz [0,0]: -0.0111 == cos(90.64 deg)

 ᴿL(V₁ → V₃)⋅Mz [1,0]: -0.9999 == sin(-89.36 deg)


_____

#### $^R\boldsymbol{S}: V_1 \to V_3$

In [6]:
## V₁ → V₃ Shear Matrix
rS_v1tov3 = applythresh(ShearMatrix_Ref(K1,eta1,C,rPv1,normal_pos=True))
print('ᴿS(V₁ → V₃) =\n',rS_v1tov3)

## del unnecessary variables
del C, K1, eta1

ᴿS(V₁ → V₃) =
 [[ 0.995301  0.004752  0.      ]
 [-0.004647  1.004699  0.      ]
 [ 0.        0.        1.      ]]


_____

_____

### $V_3 \to V_2$ Transformation

#### $^R\boldsymbol{L}: V_3 \to V_2$

In [7]:
## V₃ → V₂ Misorientation

## twinning elements
C =  rotationmatrix([0,0,1],np.pi/2) #correspondence matrix
K1 = [1,q4,0] #invariant plane
eta1 = [-q4,1,0] #shear direction

## Rotational transformation in reference frame: V₃ → V₂
rL_v3tov2 = OR_RefFrame(K1,eta1,C,rPv3,normal_pos=True)

## Print required outputs
print('ᴿL(V₃ → V₂) =\n', rL_v3tov2) #Orientation relationship
print('\n ᴿL(V₃ → V₂) [0,0]: %.4f == cos(%.2f deg)' % (rL_v3tov2[0,0],np.degrees(np.arccos(rL_v3tov2[0,0])))) #rotation angle
print('\n ᴿL(V₃ → V₂) [1,0]: %.4f == sin(%.2f deg)' % (rL_v3tov2[1,0],np.degrees(np.arcsin(rL_v3tov2[1,0])))) #rotation angle

ᴿL(V₃ → V₂) =
 [[ 0.007987 -0.999968  0.      ]
 [ 0.999968  0.007987  0.      ]
 [ 0.        0.        1.      ]]

 ᴿL(V₃ → V₂) [0,0]: 0.0080 == cos(89.54 deg)

 ᴿL(V₃ → V₂) [1,0]: 1.0000 == sin(89.54 deg)


_____

#### $^R\boldsymbol{S}: V_3 \to V_2$

In [8]:
## V₃ → V₂ Shear Matrix
rS_v3tov2 = applythresh(ShearMatrix_Ref(K1,eta1,C,rPv3,normal_pos=True))
print('ᴿS(V₃ → V₂) =\n', rS_v3tov2)

#del unnecessary variables
del C, K1, eta1

ᴿS(V₃ → V₂) =
 [[ 1.004649 -0.014481  0.      ]
 [ 0.001493  0.995351  0.      ]
 [ 0.        0.        1.      ]]


_____

_____

### $V_2 \to V_4$ Transformation

#### $^R\boldsymbol{L}: V_2 \to V_4$

In [9]:
## V₂ → V₄ Misorientation

## twinning elements
C =  rotationmatrix([1,1,0],np.pi) #correspondence matrix
K1 = [1,1,0] #invariant plane
eta1 = [-1,1,0] #shear direction

## Rotational transformation in reference frame: V₂ → V₄
rL_v2tov4 = OR_RefFrame(K1,eta1,C,rPv2,normal_pos=False)

## Print required outputs
print('ᴿL(V₂ → V₄) =\n', rL_v2tov4) #Orientation relationship
print('\n ᴿL(V₂ → V₄) [0,0]: %.4f == cos(%.2f deg)' % (rL_v2tov4[0,0],np.degrees(np.arccos(rL_v2tov4[0,0])))) #rotation angle
print('\n ᴿL(V₂ → V₄) [1,0]: %.4f == sin(%.2f deg)' % (rL_v2tov4[1,0],np.degrees(np.arcsin(rL_v2tov4[1,0])))) #rotation angle

ᴿL(V₂ → V₄) =
 [[ 0.027097  0.999633  0.      ]
 [ 0.999633 -0.027097  0.      ]
 [ 0.        0.       -1.      ]]

 ᴿL(V₂ → V₄) [0,0]: 0.0271 == cos(88.45 deg)

 ᴿL(V₂ → V₄) [1,0]: 0.9996 == sin(88.45 deg)


_____

#### Equivalent Description of $\ ^R\boldsymbol{L}: V_2 \to V_4$

$^R\boldsymbol{L}\ \simeq\ ^R\boldsymbol{L}\ \cdot\ ^R\boldsymbol{M}_z$

where, $^R\boldsymbol{M}_z$ is the mirror operator about $[001]_R$.

In [10]:
## Equivalent description of V₂ → V₄ misorientation
## ᴿL (V₂ → V₄) ⋅ Mz
dummy = rL_v2tov4 @ M_z #Orientation relationship

## Print required output
print('ᴿL(V₂ → V₄) ⋅ Mz = \n', dummy)
print('\n ᴿL(V₂ → V₄)⋅Mz [0,0]: %.4f == cos(%.2f deg)' % (dummy[0,0],np.degrees(np.arccos(dummy[0,0])))) #rotation angle
print('\n ᴿL(V₂ → V₄)⋅Mz [1,0]: %.4f == sin(%.2f deg)' % (dummy[1,0],np.degrees(np.arcsin(dummy[1,0])))) #rotation angle

## del unnecessary variables
del dummy

ᴿL(V₂ → V₄) ⋅ Mz = 
 [[ 0.027097  0.999633  0.      ]
 [ 0.999633 -0.027097  0.      ]
 [ 0.        0.        1.      ]]

 ᴿL(V₂ → V₄)⋅Mz [0,0]: 0.0271 == cos(88.45 deg)

 ᴿL(V₂ → V₄)⋅Mz [1,0]: 0.9996 == sin(88.45 deg)


_____

#### $^R\boldsymbol{S}: V_2 \to V_4$

In [11]:
## Shear matrix: V₂ → V₄
rS_v2tov4 = applythresh(ShearMatrix_Ref(K1,eta1,C,rPv2,normal_pos=False))
print('ᴿS(V₂ → V₄) =\n', rS_v2tov4)

#del unnecessary variables
del C, K1, eta1

ᴿS(V₂ → V₄) =
 [[ 0.995302 -0.004572  0.      ]
 [ 0.004827  1.004698  0.      ]
 [ 0.        0.        1.      ]]


_____

_____

### $V_4 \to V_1$ Transformation

#### $^R\boldsymbol{L}: V_4 \to V_1$

In [12]:
## V₄ → V₁ Misorientation

## twinning elements
C =  rotationmatrix([0,0,1],np.pi/2) #correspondence matrix
K1 = [1,q4,0] #invariant plane
eta1 = [-q4,1,0] #shear direction

## Rotational transformation in reference frame: V₄ → V₁
rL_v4tov1 = OR_RefFrame(K1,eta1,C,rPv4,normal_pos=True)

## Print required outputs
print('ᴿL(V₄ → V₁) =\n', rL_v4tov1) #Orientation relationship
print('\n ᴿL(V₄ → V₁) [0,0]: %.4f == cos(%.2f deg)' % (rL_v4tov1[0,0],np.degrees(np.arccos(rL_v4tov1[0,0])))) #rotation angle
print('\n ᴿL(V₄ → V₁) [1,0]: %.4f == sin(%.2f deg)' % (rL_v4tov1[1,0],np.degrees(np.arcsin(rL_v4tov1[1,0])))) #rotation angle

ᴿL(V₄ → V₁) =
 [[ 0.007987  0.999968  0.      ]
 [-0.999968  0.007987  0.      ]
 [ 0.        0.        1.      ]]

 ᴿL(V₄ → V₁) [0,0]: 0.0080 == cos(89.54 deg)

 ᴿL(V₄ → V₁) [1,0]: -1.0000 == sin(-89.54 deg)


_____

#### $^R\boldsymbol{S}: V_4 \to V_1$

In [13]:
## Shear matrix: V₄ → V₁
rS_v4tov1 = applythresh(ShearMatrix_Ref(K1,eta1,C,rPv4,normal_pos=True))
print('ᴿS(V₄ → V₁) =\n', rS_v4tov1)

ᴿS(V₄ → V₁) =
 [[ 1.004894  0.014299  0.      ]
 [-0.001675  0.995106  0.      ]
 [ 0.        0.        1.      ]]


_____

_____

### Product of Misorientation

$^R\boldsymbol{L} (V_4 \to V_1)\ \cdot\ ^R\boldsymbol{L} (V_2 \to V_4)\ \cdot\ ^R\boldsymbol{L} (V_3 \to V_2)\ \cdot\ ^R\boldsymbol{L} (V_1 \to V_3)$

In [14]:
applythresh(rL_v4tov1 @ rL_v2tov4 @ rL_v3tov2 @ rL_v1tov3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

_____

_____

### Product of Shear Matrix

$^R\boldsymbol{S} (V_4 \to V_1)\ \cdot\ ^R\boldsymbol{S} (V_2 \to V_4)\ \cdot\ ^R\boldsymbol{S} (V_3 \to V_2)\ \cdot\ ^R\boldsymbol{S} (V_1 \to V_3)$

In [15]:
applythresh(rS_v4tov1 @ rS_v2tov4 @ rS_v3tov2 @ rS_v1tov3)

array([[ 1.000206e+00, -5.959271e-06,  0.000000e+00],
       [-3.295146e-06,  9.997938e-01,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+00]])

_____

_____